In [1]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.pipeline import Pipeline
import seaborn as sns
from sklearn.base import BaseEstimator
from sklearn.compose import ColumnTransformer 
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

In [53]:
#read csv file
df=pd.read_csv('train.csv', sep=';') 

In [54]:
# Separate target and source variables
y = df.fraud
X = df.drop(['fraud','claim_amount'], axis=1)
# The 20 % of data is test size of the data 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)

# There are several features that are in reality a date
claim_date_registered claim_date_occured claim_vehicle_date_inuse 	
policy_date_start 	policy_date_next_expiry
policy_date_last_renewed
# There are features that are year of birth
policy_holder_year_birth driver_year_birth third_party_1_year_birth
third_party_2_year_birth
third_party_3_year_birth
# seems to be unique
driver_expert_id third_party_1_id driver_vehicle_id claim_vehicle_id policy_holder_id driver_id
third_party_1_vehicle_id
third_party_1_expert_id
third_party_2_id
third_party_2_vehicle_id
third_party_2_expert_id
third_party_3_id 	
third_party_3_vehicle_id 	third_party_3_expert_id
repair_id
# There is also a feature that is a time 
X_train.hist(column='claim_time_occured')

In [4]:
# See number and types of features
print("The number of features is ", X_train.shape[1])
X_train.dtypes.value_counts()

The number of features is  76


object     42
float64    25
int64       9
dtype: int64

In [5]:
# percentage of missing values
(X_train.isna().sum()/X_train.shape[0]*100).nlargest(40)

third_party_3_expert_id       100.000000
third_party_3_year_birth       99.988731
third_party_3_vehicle_id       99.970701
third_party_3_vehicle_type     99.961686
third_party_3_postal_code      99.957178
third_party_3_id               99.954924
third_party_3_injured          99.954924
third_party_3_form             99.954924
third_party_3_country          99.954924
third_party_2_expert_id        99.945909
repair_year_birth              99.851251
third_party_1_expert_id        98.798738
claim_alcohol                  98.379536
third_party_2_year_birth       97.013748
third_party_2_vehicle_id       94.746450
third_party_2_vehicle_type     93.662384
third_party_2_postal_code      92.571557
third_party_2_id               92.337165
third_party_2_injured          92.337165
third_party_2_form             92.337165
third_party_2_country          92.337165
claim_time_occured             88.334460
policy_holder_expert_id        82.672977
third_party_1_year_birth       82.206446
driver_expert_id

In [6]:
morethan20pctmiss = [element for element in (X_train.isna().sum()/X_train.shape[0]*100) if element > 20]
print("Number of features with more than 20% missing values is = ", len(morethan20pctmiss)) 

Number of features with more than 20% missing values is =  38


In [ ]:
# scatter matrix for numeric features (takes too much time)

#sns.set_theme(style="ticks")
#dfnum = df.select_dtypes(np.number)
#df1 = dfnum.assign(fraud=y.values)

#sns.pairplot(df1, hue="fraud")

In [31]:
# Create encoder to have year, month, date values
class DateTypeEncoder(BaseEstimator):

    def __init__(self):
        pass

    def fit(self, documents, y=None):
        return self

    def transform(self, x_dataset):
        x_dataset['claim_date_registered_year'] = pd.to_datetime(x_dataset['claim_date_registered']
        return x_dataset['claim_date_registered_year']


SyntaxError: invalid syntax (<ipython-input-31-6a8db1a7c7c5>, line 12)

In [58]:
from sklearn.base import TransformerMixin, BaseEstimator 

class DateTransformer(TransformerMixin, BaseEstimator):
    """Extracts features from datetime column
    
    Returns:
      hour: hour
      day: Between 1 and the number of days in the month
      month: Between 1 and 12 inclusive.
      year: four-digit year
      weekday: day of the week as an integer. Mon=0 and Sun=6
   """
    def fit(self, x, y=None):
        return self
    def transform(self, x, y=None):
        result = pd.DataFrame(x, columns=['date_hour'])
        
        result['month'] = [dt.month for dt in pd.to_datetime.result['date_hour']]
        result['year'] = [dt.year for dt in pd.to_datetime.result['date_hour']]
        result['weekday'] = [dt.weekday() for dt in pd.to_datetime.result['date_hour']]
        return result[['month', 'year', 'weekday']]
    
    def get_feature_names(self):
        return ['month', 'year', 'weekday']

In [62]:
DateTransformer =DateTransformer()
ct = ColumnTransformer(remainder='passthrough',
          transformers=[
              ('claim_date_registered', DateTransformer, ['claim_date_registered']),
              ('claim_date_occured', DateTransformer, ['claim_date_occured'])
          ])


In [63]:
model_pipeline = Pipeline(steps=[('ct',ct),
                                 ('random_forest', RandomForestRegressor(max_depth=10,random_state=0))
                                 ])

In [64]:
model_pipeline.fit(X_train,y_train)

AttributeError: 'function' object has no attribute 'result'

In [ ]:
#         x_dataset['claim_date_registered_year'] = (pd.to_datetime(x_dataset['claim_date_registered'], format='%Y%m%d')).dt.year
#         x_dataset['claim_date_registered_month'] = (pd.to_datetime(x_dataset['claim_date_registered'], format='%Y%m%d')).dt.month
#         x_dataset['claim_date_registered_day'] = (pd.to_datetime(x_dataset['claim_date_registered'], format='%Y%m%d')).dt.day
#         x_dataset['claim_date_registered_dayofweek'] = (pd.to_datetime(x_dataset['claim_date_registered'], format='%Y%m%d')).dt.dayofweek
       
 
#         x_dataset['claim_date_registered_month'] = df['claim_date_registered'].astype(str).str[4:6].astype(int)
#         x_dataset['claim_date_registered_date'] = df['claim_date_registered'].astype(str).str[6:9].astype(int)
        
#         x_dataset['claim_date_occured_year'] = df['claim_date_occured'].astype(str).str[:4].astype(int)
#         x_dataset['claim_date_occured_month'] = df['claim_date_occured'].astype(str).str[4:6].astype(int)
#         x_dataset['claim_date_occured_date'] = df['claim_date_occured'].astype(str).str[6:9].astype(int)

#         x_dataset['claim_vehicle_date_inuse_year'] = df['claim_vehicle_date_inuse'].astype(str).str[:4].astype(int)
#         x_dataset['claim_vehicle_date_inuse_month'] = df['claim_vehicle_date_inuse'].astype(str).str[4:6].astype(int)
#         x_dataset['claim_vehicle_date_inuse_date'] = df['claim_vehicle_date_inuse'].astype(str).str[6:9].astype(int)

#         x_dataset['policy_date_start_year'] = df['policy_date_start'].astype(str).str[:4].astype(int)
#         x_dataset['policy_date_start_month'] = df['policy_date_start'].astype(str).str[4:6].astype(int)
#         x_dataset['policy_date_start_date'] = df['policy_date_start'].astype(str).str[6:9].astype(int)

#         x_dataset['policy_date_next_expiry_year'] = df['policy_date_next_expiry'].astype(str).str[:4].astype(int)
#         x_dataset['policy_date_next_expiry_month'] = df['policy_date_next_expiry'].astype(str).str[4:6].astype(int)
#         x_dataset['policy_date_next_expiry_date'] = df['policy_date_next_expiry'].astype(str).str[6:9].astype(int)

#         x_dataset['policy_date_last_renewed_year'] = df['policy_date_last_renewed'].astype(str).str[:4].astype(int)
#         x_dataset['policy_date_last_renewed_month'] = df['policy_date_last_renewed'].astype(str).str[4:6].astype(int)
#         x_dataset['policy_date_last_renewed_date'] = df['policy_date_last_renewed'].astype(str).str[6:9].astype(int)
      

In [9]:
# Create encoder to pinpoint missinvalues in different columns
class MissingColumnEncoder(BaseEstimator):

    def __init__(self):
        pass

    def fit(self, documents, y=None):
        return self

    def transform(self, x_dataset):
        for col in x_dataset.columns:
            x_dataset[col+'_miss'] = x_dataset[col].isna()*1
            return x_dataset

In [10]:
# Create encoder to get age features
class AgeEncoder(BaseEstimator):

    def __init__(self):
        pass

    def fit(self, documents, y=None):
        return self

    def transform(self, x_dataset):
        x_dataset['policy_holder_year_birth'] = 2021-x_dataset['policy_holder_year_birth'] 
        x_dataset['driver_year_birth'] = 2021-x_dataset['driver_year_birth'] 
        x_dataset['third_party_1_year_birth'] = 2021-x_dataset['third_party_1_year_birth'] 
        x_dataset['third_party_2_year_birth'] = 2021-x_dataset['third_party_2_year_birth'] 
        x_dataset['third_party_3_year_birth'] = 2021-x_dataset['third_party_3_year_birth'] 
        x_dataset['repair_year_birth'] = 2021-x_dataset['repair_year_birth']
        
        return x_dataset

In [11]:
pre_process = ColumnTransformer(remainder='passthrough',
                                transformers=[('drop_columns', 'drop', ['claim_vehicle_date_inuse',                                                                         
                                                                        'policy_date_start',
                                                                        'policy_date_next_expiry',
                                                                        'policy_date_last_renewed',
                                                                        'claim_date_registered ',
                                                                        'claim_date_occured',
                                                                                                                                                
                                                                       ]),
                                              ('impute_missing_cat', SimpleImputer(strategy='most_frequent'), ['claim_postal_code',
                                                                                                           'claim_cause',
                                                                                                           'claim_liable',
                                                                                                           'claim_num_vehicles',
                                                                                                           'claim_police',
                                                                                                           'claim_alcohol',
                                                                                                           'claim_language',
                                                                                                           'claim_vehicle_id',
                                                                                                           'claim_vehicle_brand',
                                                                                                           'claim_vehicle_type',
                                                                                                           'claim_vehicle_fuel_type',
                                                                                                           'policy_holder_id',
                                                                                                           'policy_holder_postal_code',
                                                                                                           'policy_holder_form',
                                                                                                           'policy_holder_country',
                                                                                                           'policy_holder_expert_id',
                                                                                                           'driver_id',
                                                                                                           'driver_postal_code',
                                                                                                           'driver_form',
                                                                                                           'driver_country',
                                                                                                           'driver_expert_id',
                                                                                                           'driver_injured',
                                                                                                           'driver_vehicle_id',
                                                                                                           'third_party_1_id',
                                                                                                           'third_party_1_postal_code',
                                                                                                           'third_party_1_injured',
                                                                                                           'third_party_1_vehicle_type',
                                                                                                           'third_party_1_form',
                                                                                                           'third_party_1_country',
                                                                                                           'third_party_1_vehicle_id',
                                                                                                           'third_party_1_expert_id',
                                                                                                           'third_party_2_id',
                                                                                                           'third_party_2_postal_code',
                                                                                                           'third_party_2_injured',
                                                                                                           'third_party_2_vehicle_type',
                                                                                                           'third_party_2_form',
                                                                                                           'third_party_2_country',
                                                                                                           'third_party_2_vehicle_id',
                                                                                                           'third_party_2_expert_id',
                                                                                                           'third_party_3_id',
                                                                                                           'third_party_3_postal_code',
                                                                                                           'third_party_3_injured',
                                                                                                           'third_party_3_vehicle_type',
                                                                                                           'third_party_3_form',
                                                                                                           'third_party_3_country',
                                                                                                           'third_party_3_vehicle_id',
                                                                                                           'third_party_3_expert_id',
                                                                                                           'repair_id',
                                                                                                           'repair_form',
                                                                                                           'repair_country',
                                                                                                           'repair_sla',
                                                                                                           'policy_coverage_type']),
                                              ('impute_missing_num', SimpleImputer(strategy='median'),['claim_time_occured',
                                                                                                       'claim_num_injured',
                                                                                                       'claim_num_third_parties',
                                                                                                       'claim_vehicle_cyl',
                                                                                                       'claim_vehicle_load',
                                                                                                       'claim_vehicle_power',
                                                                                                       'policy_holder_year_birth',
                                                                                                       'driver_year_birth',
                                                                                                       'third_party_1_year_birth',
                                                                                                       'third_party_2_year_birth',
                                                                                                       'third_party_3_year_birth',
                                                                                                       'repair_year_birth',
                                                                                                       'policy_num_changes',
                                                                                                       'policy_num_claims',
                                                                                                       'policy_premium_100',
                                                                                                       'policy_coverage_1000',
                                                                                                       ])])

In [12]:
model_pipeline = Pipeline(steps=[('get_year_month_date', DateTypeEncoder()),
                                 ('get_columns_for_missing',MissingColumnEncoder()),
                                 ('get_age_from_year_birth',AgeEncoder()),
                                 ('pre_processing',pre_process),
                                 ('random_forest', RandomForestRegressor(max_depth=10,random_state=0))
                                 ])

In [ ]:
ct = ColumnTransformer(
          transformers=[
              (‘dates1’, DateTransformer, [‘claim_date_registered’]),
              (‘dates2’, DateTransformer, [‘end_date’])
          ])

In [25]:
model_pipeline = Pipeline(steps=['get_year_month_date', DateTypeEncoder(),
                                 ('random_forest', RandomForestRegressor(max_depth=10,random_state=0))
                                 ])

TypeError: 'DateTypeEncoder' object is not iterable

In [15]:
model_pipeline.fit(X_train,y_train)

<ipython-input-9-85742b2e98cf>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_dataset[col+'_miss'] = x_dataset[col].isna()*1
<ipython-input-10-804feea8fda7>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_dataset['policy_holder_year_birth'] = 2021-x_dataset['policy_holder_year_birth']
<ipython-input-10-804feea8fda7>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

ValueError: A given column is not a column of the dataframe